## Automating Global Geospatial Data Set Analysis
### Visualizing flood disasters in the cities of the Global South

This code will turn raster data into an interactive local kepler.gl web map displaying the extent of flooding events and its effects on the local population and infrastructure.

The script is used as the research method in the Master's thesis of Ohto Nygren for the Department of Geography at the University of Turku, Finland.

Flood data has been provided by the new space company ICEYE and the world settlement footprint (WSF) population data that was used was provided by the German Aerospace Agency (DLR). The script has been optimized for these datasets, but similar data can most likely be used with the script.

The OSMnx module is used to pull data from OpenStreetMap (OSM) to get estimations of inundated buildings, with an emphasis on hospitals and pharmacies.

Estimations provided by the script are not completely accurate due to the lack of mapped OSM data, but flood depths captured by ICEYE's SAR satellites are very accurate and the population data created by the DLR is very accurate for a global dataset. 

The main idea is to provide quick first estimations via Python automatization of the extent of flood damage and the number of people affected by a flooding event to first responders and international humanitarian aid.

Conctact information for further questions:

Ohto Nygren

omnygr@utu.fi

MSc student in geography

University of Turku

Finland

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import osmnx as ox
import shapely
from shapely.geometry import Point, LineString, Polygon, shape
from shapely.geometry import box
from shapely import speedups
import rasterio as rio
import rasterio.features
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.features import shapes
from rasterio.mask import mask
from fiona.crs import from_epsg
import pycrs
import os
import mapclassify
import seaborn as sns
import utm
from pyproj import CRS

### Raster transformations

In [ ]:
# Filepath for WSF raster file
wsf_fp = r'/Users/ohtonygren/Yliopisto/Gradu/Data/WSF/Bangkok_WSF2019population_10m.tif'
wsf_raster = rio.open(wsf_fp)

# Filepath for ICEYE raster file
iceye_fp = r'/Users/ohtonygren/Yliopisto/Gradu/Data/ICEYE/Bangkok_iceye.tif'
iceye_orig = rio.open(iceye_fp)

from rasterio.warp import calculate_default_transform, reproject, Resampling

# Getting CRS from the WSF data
dstCrs = wsf_raster.crs

# Calculate transform array and shape of reprojected layer
transform, width, height = calculate_default_transform(iceye_orig.crs, dstCrs, iceye_orig.width, iceye_orig.height, *iceye_orig.bounds)

# Working of the meta for the destination raster
kwargs = iceye_orig.meta.copy()
kwargs.update({'crs': dstCrs, 'transform': transform, 'width': width, 'height': height})

iceye_raster = rio.open(iceye_fp, 'w', **kwargs)

# Reproject and save raster band data
for i in range(1, iceye_raster.count +1):
    reproject(
        source=rio.band(iceye_orig, i),
        destination=rio.band(iceye_raster, i),
        src_crs=iceye_raster.crs,
        dstCrs=dstCrs,
        resampling=Resampling.nearest)

# Close destination raster    
iceye_raster.close()

print('Progress: Raster reprojection done.')

# Reopening the raster that is now projected to EPSG: 4326
iceye_wgs84 = rio.open(iceye_fp)

### Vector transformations

In [ ]:
# Polygonizing the raster file
mask = None
with rio.Env():
    with rio.open(iceye_fp) as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)

# Creating a GeoDataFrame from the polygonized raster
iceye_breaks = gpd.GeoDataFrame.from_features(geoms)

# Get indexes where 'raster_val' column has value 'min'
indexNames = iceye_breaks[iceye_breaks['raster_val'] == iceye_breaks['raster_val'].min()].index
# Delete 'min' rows indexes from dataframe as they are Null values in the raster file
iceye_breaks.drop(indexNames, inplace=True)

print('Progress: Polygonizing done.')

# Rounding up unnecessary decimals
iceye_breaks['raster_val'] = iceye_breaks['raster_val'].round(3)

# Removing all rows with the value of zero
iceye_breaks[iceye_breaks.raster_val != 0.000]

# Classifying the data by flood hazard levels
breaks = mapclassify.UserDefined(iceye_breaks['raster_val'], bins=[0.1, 0.25, 0.5, 1, 2, 2.5])
iceye_breaks['flood_class'] = iceye_breaks[['raster_val']].apply(breaks)

# Dissolving the data into the 6 classified breaks
iceye_breaks = iceye_breaks.dissolve(by='flood_class', as_index=False)

### Raster clipping and affected population estimation

In [ ]:
# Modified from: Lau & Um (2021) Using GeoSpatial Data Analytics: A Friendly Guide to Folium and Rasterio
# URL: https://omdena.com/blog/geospatial-data-analytics/
#
# ------------------

# Creating an empty list for storing values
results = []

# Looping through flood classes in dataframe and using them as masks for extracting values
for i in iceye_breaks['flood_class']:
        
    roi = iceye_breaks[iceye_breaks.flood_class == i]

    # Using the mask.mask Rasterio module for specifying the ROI
    gtraster, bound = rio.mask.mask(wsf_raster, roi['geometry'], crop=True)
    
    # Using values greater than 0 to get population data from the WSF2019-Pop raster pixels
    results.append(gtraster[0][gtraster[0]>0].sum())

# Saving results in new column
iceye_breaks['population'] = results

# Dividing by 1000 because the WSF2019-Pop data has been multiplied by 1000 to save the file as integer
iceye_breaks['population'] = iceye_breaks['population'].div(1000)

#### Getting OSM data from flooded area

In [ ]:
# Splitting the flood areas into > 1m and < 1m flood depth areas
flood = iceye_breaks.from_features(iceye_breaks[0:3])
severe_flood = iceye_breaks.from_features(iceye_breaks[3:7])

# Creating a constant value to use for dissolve
iceye_breaks['dissolve'] = 1
flood['dissolve'] = 1

# Dissolving all the flooded polygons into flooding extents for the geometries_from_polygon feature
flood_aoi = iceye_breaks.dissolve(by='dissolve')
flood_area = flood.dissolve(by='dissolve')

# Turning the GeoDataFrames into a shapely polygons for OSMnx
osm_aoi = flood_aoi.iloc[0]['geometry']
flood_shape = flood_area.iloc[0]['geometry']
sos_aoi = severe_flood.iloc[0]['geometry']

# Getting OpenStreetMap data from the flooded area
hospital = ox.geometries.geometries_from_polygon(osm_aoi, tags={'building':'hospital', 'amenity':'hospital'})
pharmacy = ox.geometries.geometries_from_polygon(osm_aoi, tags={'building':'pharmacy', 'amenity':'pharmacy'})
sos_build = ox.geometries.geometries_from_polygon(sos_aoi, tags={'building': True})
low_flood = ox.geometries.geometries_from_polygon(flood_shape, tags={'building': True})
buildings = ox.geometries.geometries_from_polygon(osm_aoi, tags={'building': True})

# Removing unnecessary columns
sos_build = sos_build[['geometry', 'building', 'name']]
low_flood = low_flood[['geometry', 'building', 'name']]
buildings = buildings[['geometry', 'building', 'name']]

# Removing buildings that are point data
sos_build = sos_build.loc[sos_build.geometry.type=='Polygon']
low_flood = low_flood.loc[low_flood.geometry.type=='Polygon']
buildings = buildings.loc[buildings.geometry.type=='Polygon']

# Removing the dissolve column
del iceye_breaks['dissolve']
del flood['dissolve']

# Reseting the index of OSM data
low_flood = low_flood.reset_index()
sos_build = sos_build.reset_index()
buildings = buildings.reset_index()

# Removing duplicate buildings from low flood area
low_flood.osmid.isin(sos_build.osmid)
~low_flood.osmid.isin(sos_build.osmid)
low_flood = low_flood[~low_flood.osmid.isin(sos_build.osmid)]

# Removing duplicate buildings
buildings.osmid.isin(sos_build.osmid)
~buildings.osmid.isin(sos_build.osmid)
buildings = buildings[~buildings.osmid.isin(sos_build.osmid)]
buildings.osmid.isin(low_flood.osmid)
~buildings.osmid.isin(low_flood.osmid)
buildings = buildings[~buildings.osmid.isin(low_flood.osmid)]

# Removing unnecessary columns
del low_flood['element_type']
del sos_build['element_type']
del buildings['element_type']

### RESULTS

In [ ]:
# Reprojecting the data to a CRS that uses a metric system instead of degrees for the buffering
hospital = hospital.to_crs(epsg=3857)
pharmacy = pharmacy.to_crs(epsg=3857)

# Calculating the centroids of all the points & polygons in the pharmacy dataset
hospital['geometry'] = hospital['geometry'].centroid
pharmacy['geometry'] = pharmacy['geometry'].centroid

# Reprojecting the data back to a CRS that is better for plotting
hospital = hospital.to_crs(epsg=4326)
pharmacy = pharmacy.to_crs(epsg=4326)

# Getting x and y coordinates of hospitals for mapping
hospital['x'] = hospital['geometry'].apply(lambda geom: geom.x)
hospital['y'] = hospital['geometry'].apply(lambda geom: geom.y)

# Getting x and y coordinates of pharmacies for mapping
pharmacy['x'] = pharmacy['geometry'].apply(lambda geom: geom.x)
pharmacy['y'] = pharmacy['geometry'].apply(lambda geom: geom.y)

# Creating a list of coordinate pairs
hospitals = list(zip(hospital['y'], hospital['x']))
pharmacies = list(zip(pharmacy['y'], pharmacy['x']))

# Adding icon columns for kepler gl visualization later on
hospital['icon'] = 'plus-alt'
pharmacy['icon'] = 'pin'

# Setting a coordinate system to the data
iceye_breaks.crs = 'EPSG:4326'

# Function modified from: https://gis.stackexchange.com/questions/429601/
def findtheutm(aGeometry):
    """A function to find a coordinates UTM zone"""
    x, y, parallell, latband = utm.from_latlon(aGeometry.centroid.y, aGeometry.centroid.x)
    if latband in 'CDEFGHJKLM': #https://www.lantmateriet.se/contentassets/379fe00e09d74fa68550f4154350b047/utm-zoner.gif
        ns = 'S'
    else:
        ns = 'N'
    crs = "+proj=utm +zone={0} +{1}".format(parallell, ns) #https://gis.stackexchange.com/questions/365584/convert-utm-zone-into-epsg-code
    crs = CRS.from_string(crs)
    _, code = crs.to_authority()
    return int(code)

epsg = findtheutm(iceye_breaks.geometry.iloc[0])

# Calculating the area of flood extents in km2
iceye_breaks['area'] = iceye_breaks.to_crs(epsg).area / 10**6

# Creating a new column
iceye_breaks['flood_depth'] = ''

# Creating labels for the flood depths of the different flood classes
iceye_breaks.loc[0, 'flood_depth'] = '0 ≥ 0.1 meters'
iceye_breaks.loc[1, 'flood_depth'] = '0.1 ≥ 0.25 meters'
iceye_breaks.loc[2, 'flood_depth'] = '0.25 ≥ 0.5 meters'
iceye_breaks.loc[3, 'flood_depth'] = '0.5 ≥ 1 meters'
iceye_breaks.loc[4, 'flood_depth'] = '1 ≥ 2 meters'
iceye_breaks.loc[5, 'flood_depth'] = '2 ≥ 2.5 meters'
iceye_breaks.loc[6, 'flood_depth'] = '2.5 ≥ meters'

# Barplot of people affected by flood
fig, ax = plt.subplots(figsize=(12,10))
sns.barplot(y='population', x='flood_depth', palette='rocket_r', data=iceye_breaks, ax=ax)
ax.set_title('Amount of people living in the flooded area according to the WSF2019-Pop data', fontsize=15)
ax.bar_label(ax.containers[0])
ax.set_ylabel('Population', fontsize=15)
ax.set_xlabel('Flood depth', fontsize=15)

plt.tight_layout()
plt.savefig('flooded_population.jpg')

# Printing some results
print(len(sos_build), 'buildings in the flooded area are severely inundated and', len(low_flood) + len(sos_build) + len(buildings), 'buildings have flooded in total according to existing OpenStreetMap data.')
print('There are',(len(pharmacy)), 'pharmacies within the flood danger area.')
print('There are',(len(hospital)),'hospitals within the flood danger area.')
print('The total flooded area is', iceye_breaks['area'].sum().round(2), 'km2.')

### Kepler.gl

In [ ]:
from keplergl import KeplerGl
map_1 = KeplerGl(height=1000)

kepler_flood = iceye_breaks.to_json()
sos_j = sos_build.to_json()
low_j = low_flood.to_json()
build_j = buildings.to_json()
hospitals = hospital.to_json()
pharmacies = pharmacy.to_json()

map_1.add_data(data=hospitals, name='Hospitals in flood area')
map_1.add_data(data=pharmacies, name='Pharmacies in flood area')
map_1.add_data(data=sos_j, name='Severely inundated buildings')
map_1.add_data(data=build_j, name='Inundated buildings')
map_1.add_data(data=low_j, name='Slightly inundated buildings')
map_1.add_data(data=kepler_flood, name='Flood depth')

# Setting a CRS that uses meters
flood_aoi.crs = 'EPSG:3857'

# Getting the lat and lon data of building vectors for automatical zooming to AOI in interactive map
flood_aoi['lon'] = flood_aoi.centroid.x
flood_aoi['lat'] = flood_aoi.centroid.y

# Getting coordinates for map zoom
lat = flood_aoi['lat'].values[0]
lon = flood_aoi['lon'].values[0]

In [ ]:
config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '6blfkxe',
     'type': 'icon',
     'config': {'dataId': 'Hospitals in flood area',
      'label': 'Hospitals in flood area',
      'color': [218, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'y', 'lng': 'x', 'icon': 'icon', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 35,
       'fixedRadius': False,
       'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': 'cacpee',
     'type': 'icon',
     'config': {'dataId': 'Pharmacies in flood area',
      'label': 'Pharmacies in flood area',
      'color': [82, 163, 83],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'y', 'lng': 'x', 'icon': 'icon', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 50,
       'fixedRadius': False,
       'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': 'fnd5s2d',
     'type': 'geojson',
     'config': {'dataId': 'Severely inundated buildings',
      'label': 'Severely inundated buildings',
      'color': [245, 153, 153],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.2,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [218, 0, 0],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'fr0h2g',
     'type': 'geojson',
     'config': {'dataId': 'Inundated buildings',
      'label': 'Inundated buildings',
      'color': [255, 152, 51],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.2,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [239, 93, 40],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'xcowcw',
     'type': 'geojson',
     'config': {'dataId': 'Slightly inundated buildings',
      'label': 'Slightly inundated buildings',
      'color': [255, 228, 102],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.2,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [246, 218, 0],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': 'e7iibuv',
     'type': 'geojson',
     'config': {'dataId': 'Flood depth',
      'label': 'Flood depth',
      'color': [77, 193, 156],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [119, 110, 87],
       'colorRange': {'name': 'ColorBrewer Blues-7',
        'type': 'singlehue',
        'category': 'ColorBrewer',
        'colors': ['#eff3ff',
         '#c6dbef',
         '#9ecae1',
         '#6baed6',
         '#4292c6',
         '#2171b5',
         '#084594']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'flood_depth',
       'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'Hospitals in flood area': [{'name': 'addr:street',
        'format': None},
       {'name': 'name', 'format': None},
       {'name': 'name:en', 'format': None}],
      'Pharmacies in flood area': [{'name': 'addr:street', 'format': None},
       {'name': 'name', 'format': None},
       {'name': 'name:en', 'format': None}],
      'Severely inundated buildings': [{'name': 'name', 'format': None}],
      'Inundated buildings': [{'name': 'name', 'format': None}],
      'Slightly inundated buildings': [{'name': 'name', 'format': None}],
      'Flood depth': [{'name': 'area', 'format': None},
       {'name': 'flood_class', 'format': None},
       {'name': 'flood_depth', 'format': None},
       {'name': 'population', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': True},
    'coordinate': {'enabled': True}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 0,
   'dragRotate': False,
   'latitude': lat,
   'longitude': lon,
   'pitch': 0,
   'zoom': 10.650982495801145,
   'isSplit': False},
  'mapStyle': {'styleType': 'satellite',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [ ]:
map_1.save_to_html(data={'Pharmacies in flood area' : pharmacies,
                         'Hospitals in flood area' : hospitals,
                         'Severely inundated buildings' : sos_j,
                         'Inundated buildings' : build_j,
                         'Slightly inundated buildings' : low_j,
                         'Flood depth' : kepler_flood},
                   config=config,
                   file_name='Kepler_flood_map.html')